In [6]:
import os
import torch
import timm
import numpy as np
import random
from sklearn.metrics import f1_score
from tqdm import tqdm
from torchvision import transforms, datasets
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler

In [7]:
# ✅ 고정 seed
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

set_seed(42)

In [8]:

# ✅ 2. 장치 설정
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("📌 DEVICE:", DEVICE)

# ✅ 3. 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# ✅ 4. 데이터셋 로딩 & 분리
dataset = datasets.ImageFolder("train_preprocessed", transform=transform)
val_ratio = 0.1
val_size = int(len(dataset) * val_ratio)
train_size = len(dataset) - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)


📌 DEVICE: cuda


In [9]:
# ✅ 5. 모델 불러오기
model = timm.create_model('mobilenetv3_large_100', pretrained=True, num_classes=7)
model.to(DEVICE)

# ✅ 6. 손실, 최적화, AMP
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scaler = GradScaler()

C:\Users\신르이철\AppData\Local\Temp\ipykernel_22872\3362017307.py:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [10]:
# ✅ 7. 학습 루프
best_f1 = 0
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0
    loop = tqdm(train_loader, desc=f"🚀 Epoch {epoch+1}/{EPOCHS}")

    for images, labels in loop:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()

        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
        loop.set_postfix(loss=train_loss / (loop.n + 1))

    # ✅ 8. 검증
    model.eval()
    val_preds, val_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(DEVICE)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            val_preds.extend(preds)
            val_labels.extend(labels.numpy())

    val_f1 = f1_score(val_labels, val_preds, average='macro')
    print(f"📊 Validation F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_mobilenetv3.pt')
        print(f"💾 Best model saved (F1: {best_f1:.4f})")

print("✅ 학습 완료")

🚀 Epoch 1/10:   0%|          | 0/10689 [00:00<?, ?it/s]C:\Users\신르이철\AppData\Local\Temp\ipykernel_22872\3991835558.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
🚀 Epoch 1/10:   2%|▏         | 164/10689 [01:29<1:36:04,  1.83it/s, loss=1.57]


KeyboardInterrupt: 